In [17]:
import numpy as np
import os, shutil, json, datetime


def is_day(full_path):
    head, fname = os.path.split(full_path)
    json_path = os.path.join(head, "ride.json")
    
    # read out the json file
    with open(json_path) as data_file:
        data = json.load(data_file)

    # find the segment's info
    the_seg = [x for x in data["segments"] if x["filename"]==fname]
    assert(len(the_seg) == 1)
    seg = the_seg[0]

    # record the time and location
    time_interval = (seg["startTime"]/1000, seg["endTime"]/1000)
    time_interval = [datetime.datetime.fromtimestamp(x) for x in time_interval]
    start = time_interval[0]
    if 6 < start.time().hour < 17:
        return True
    else:
        return False

full_video_list_fname = "/data/nx-bdd-20160929/video_filtered_index_38_60_sec.txt"
tfrecord_path = "/data/nx-bdd-20160929/tfrecord_fix_speed/train/"
num_videos = 100
output_path = "/data/yang/data/opensfm"

with open(full_video_list_fname, "r") as f:
    full_video_list = f.readlines()
    
name2path = {}
for path in full_video_list:
    path = path.strip()
    head, tail = os.path.split(path)
    name2path[tail] = path

# get the list of files in "/data/nx-bdd-20160929/tfrecord_fix_speed"
ivideo = 0
for file in os.listdir(tfrecord_path):
    video_hash = file.split(".")[0]
    video_name = video_hash + ".mov"
    full_path = name2path[video_name]
    
    if not is_day(full_path):
        continue
    
    # copy the video & ride.json to the opensfm directory
    target_dir = os.path.join(output_path, video_hash)
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    target_video = os.path.join(target_dir, video_name)
    if not os.path.exists(target_video):
        shutil.copyfile(full_path, target_video)
    head, tail = os.path.split(full_path)
    shutil.copyfile(os.path.join(head, "ride.json"),
                    os.path.join(target_dir, "ride.json"))
    
    ivideo += 1
    if ivideo == num_videos:
        break